In [4]:
from py2neo import Graph,Node,Relationship

## 1 Graph
在 database 模块中包含了和 Neo4j 数据交互的 API，最重要的当属 Graph，它代表了 Neo4j 的图数据库，同时 Graph 也提供了许多方法来操作 Neo4j 数据库。

In [5]:
graph=Graph(
    "http://localhost:7474",
    username="neo4j",
    password="miffy1314"
)

## 1.1 建立节点

In [6]:
a=Node('Person',name='Alice')
b=Node('Person',name='Bob')
r=Relationship(a,'KNOWS',b)
s=a|b|r
print(s)
graph=graph.create(s)

({(alice:Person {name:"Alice"}), (bob:Person {name:"Bob"})}, {(alice)-[:KNOWS]->(bob)})


In [ ]:
# 方式 2
a = Node('Person', name='Alice')
test_graph.create(a)
b = Node('Person', name='Bob')
test_graph.create(b)
ab = Relationship(a, 'KNOWS', b)
graph.create(ab)

## 1.2 查询结果
- 利用 data() 方法来获取查询结果

In [27]:
data=graph.data('MATCH (p:Person) return p')
print(data)

[{'p': (alice:Person {name:"Alice"})}, {'p': (bob:Person {name:"Bob"})}]


输出结果即 CQL 语句的返回结果，是列表形式

In [30]:
from pandas import DataFrame
df=DataFrame(data)
print(df)

                   p
0  {'name': 'Alice'}
1    {'name': 'Bob'}


- 使用 find_one() 或 find() 方法进行 Node 的查找，
- 利用 match() 或 match_one() 方法对 Relationship 进行查找

In [31]:
node = graph.find_one(label='Person')
print(node)
relationship = graph.match_one(rel_type='KNOWS')
print(relationship)

(alice:Person {name:"Alice"})
(alice)-[:KNOWS]->(bob)


G:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  """Entry point for launching an IPython kernel.


- 使用 push() 方法更新Node的属性

In [32]:
a = Node('Person', name='Alice')
node = graph.find_one(label='Person')
node['age'] = 21
graph.push(node)
print(graph.find_one(label='Person'))

G:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  


(alice:Person {age:21,name:"Alice"})


G:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  """


- 使用 delete() 方法删除某个 Node

在删除 Node 时必须先删除其对应的 Relationship，否则无法删除 Node

In [33]:
node = graph.find_one(label='Person')
relationship = graph.match_one(rel_type='KNOWS')
graph.delete(relationship)
graph.delete(node)

G:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead
  """Entry point for launching an IPython kernel.


## 2 NodeSelector

Graph 有时候用起来不太方便，比如如果要根据多个条件进行 Node 的查询是做不到的，在这里更方便的查询方法是利用 NodeSelector，我们首先新建如下的 Node 和 Relationship，实例如下

In [34]:
a = Node('Person', name='Alice', age=21, location='广州')
b = Node('Person', name='Bob', age=22, location='上海')
c = Node('Person', name='Mike', age=21, location='北京')
r1 = Relationship(a, 'KNOWS', b)
r2 = Relationship(b, 'KNOWS', c)
graph.create(a)
graph.create(r1)
graph.create(r2)

- 在这里我们用 NodeSelector 来筛选 age 为 21 的 Person Node

In [35]:
from py2neo import  NodeSelector
selector = NodeSelector(graph)
persons = selector.select('Person', age=21)
print(list(persons))

[(mike:Person {age:21,location:"北京",name:"Mike"}), (alice:Person {age:21,location:"广州",name:"Alice"})]


- 可以使用 where() 进行更复杂的查询，例如查找 name 是 A 开头的 Person Node

In [36]:
selector = NodeSelector(graph)
persons = selector.select('Person').where('_.name =~ "A.*"')
print(list(persons))

[(alice:Person {age:21,location:"广州",name:"Alice"})]


- 使用 order_by() 进行排序

In [37]:
selector = NodeSelector(graph)
persons = selector.select('Person').order_by('_.age')
print(list(persons))

[(mike:Person {age:21,location:"北京",name:"Mike"}), (alice:Person {age:21,location:"广州",name:"Alice"}), (bob:Person {age:22,location:"上海",name:"Bob"})]


- 前面返回的都是列表，如果要查询单个节点的话，可以使用 first() 方法

In [38]:
selector = NodeSelector(graph)
person = selector.select('Person').where('_.name =~ "A.*"').first()
print(person)

(alice:Person {age:21,location:"广州",name:"Alice"})
